<a href="https://colab.research.google.com/github/runnin123/Jeju_Bigdata/blob/master/Jeju_region_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from itertools import product

# 평가 메트릭
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 통계
from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [4]:
class Model:
    def __init__(self, data, num):
        self.X_train = data[0]
        self.X_test = data[1]
        self.y_train = data[2]
        self.y_test = data[3]
        self.encoding_data = num

    def rmsle(self, y, pred): 
        log_y = np.log1p(y)
        log_pred = np.log1p(pred)
        squared_error = (log_y - log_pred)**2
        rmsle = np.sqrt(np.mean(squared_error))
        print(round(rmsle, 3))
#       print('Test Data RMSLE: {0.3f}'.format(rmsle))
        return round(rmsle, 3)

    def best_params_model(self, model, params):
        cv_model = GridSearchCV(model, param_grid=params, scoring="neg_mean_squared_error", cv = 5)
        cv_model.fit(self.X_train, self.y_train)

        print("----", model.__class__.__name__, "----")
        print("GridSearchCV 최적 하이퍼 파라미터 :", cv_model.best_params_)

        rmse = np.sqrt(-1*cv_model.best_score_)
        print("GridSearchCV 최적 평균 RMSE값 :", np.round(rmse, 3))

        eval_pred = cv_model.predict(self.X_test)
        eval_pred = np.expm1(eval_pred)
        rmsle_ = self.rmsle(self.y_test, eval_pred)

        return  cv_model.best_estimator_, rmsle_

    def get_model(self):
        xgb = XGBRegressor(random_state=0)
        gbm = GradientBoostingRegressor(random_state=0)
        lgb = LGBMRegressor(random_state=0)

        params = {'n_estimators': [1000, 2000]}

        models = [xgb, gbm, lgb]
        best_models = []
        rmsles = []
        
        for model in models:
            new_model = self.best_params_model(model, params)[0]
            new_rmsle = self.best_params_model(model, params)[1]
            best_models.append(new_model)
            rmsles.append(new_rmsle)

        self.xgb_reg = best_models[0]
        self.gbm_reg = best_models[1]
        self.lgb_reg = best_models[2]

        self.xgb_rmsle = rmsles[0]
        self.gbm_rmsle = rmsles[1]
        self.lgb_rmsle = rmsles[2]

    def final(self):

        xgb_pred = self.xgb_reg.predict(self.X_test)
        xgb_pred = np.expm1(xgb_pred)

        gbm_pred = self.gbm_reg.predict(self.X_test)
        gbm_pred = np.expm1(gbm_pred)

        lgb_pred = self.lgb_reg.predict(self.X_test)
        lgb_pred = np.expm1(lgb_pred)

        pred = np.array([xgb_pred, gbm_pred, lgb_pred])
        pred = np.transpose(pred)

        rmsle_sum = self.xgb_rmsle + self.gbm_rmsle + self.lgb_rmsle

        self.xgb_per = self.xgb_rmsle / rmsle_sum
        self.gbm_per = self.gbm_rmsle / rmsle_sum
        self.lgb_per = self.lgb_rmsle / rmsle_sum
        
        final = xgb_pred*self.xgb_per + gbm_pred*self.gbm_per + lgb_pred*self.lgb_per
      # print("The Final rmsle")
        self.rmsle(self.y_test, final)

    def make_temp(self):
        CARD_SIDO_NMs = self.encoding_data['CARD_SIDO_NM'].unique()
        STD_CLSS_NMs  = self.encoding_data['STD_CLSS_NM'].unique()
        HOM_SIDO_NMs  = self.encoding_data['HOM_SIDO_NM'].unique()
        AGEs          = self.encoding_data['AGE'].unique()
        SEX_CTGO_CDs  = self.encoding_data['SEX_CTGO_CD'].unique()
        FLCs          = self.encoding_data['FLC'].unique()
        years         = [2020]
        months        = [4, 7]

        comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs, HOM_SIDO_NMs, AGEs, SEX_CTGO_CDs, FLCs, years, months]
        temp = np.array(list(product(*comb_list)))

        train_features = self.encoding_data.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
        tmp = pd.DataFrame(data=temp, columns=train_features.columns)

        return tmp

    def make_sub(self, temp):
        xgb_pred = self.xgb_reg.predict(temp)
        xgb_pred = np.expm1(xgb_pred)

        gbm_pred = self.gbm_reg.predict(temp)
        gbm_pred = np.expm1(gbm_pred)

        lgb_pred = self.lgb_reg.predict(temp)
        lgb_pred = np.expm1(lgb_pred)

        final_rmsle = xgb_pred*self.xgb_per + gbm_pred*self.gbm_per + lgb_pred*self.lgb_per

        temp['AMT'] = np.round(final_rmsle, 0)
        temp['REG_YYMM'] = temp['year']*100 + temp['month']
        temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
        temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop = False)

        temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
        temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

        return temp

In [5]:
def sampling():
    project_id = 'jeju-bigquery-282708'
    client = bigquery.Client(project=project_id)

    train = client.query('''
    SELECT 
        * 
    FROM `jeju-bigquery-282708.jeju_bigdata.201901_202003_train`
    ''').to_dataframe()

    return train

In [6]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [7]:
def data_pre(data):
    data = data.fillna('')
    data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
    data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
    data = data.drop(['REG_YYMM'], axis = 1)

    return data

In [8]:
def cate(data):
    local = data[(data['CARD_SIDO_NM'] == data['HOM_SIDO_NM']) & (data['CARD_CCG_NM'] == data['HOM_CCG_NM'])].reset_index(drop = True)
    tourist = data[(data['CARD_SIDO_NM'] != data['HOM_SIDO_NM']) | (data['CARD_CCG_NM'] != data['HOM_CCG_NM'])].reset_index(drop = True)
    
    local, tourist = local.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1), tourist.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)
    columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
    local, tourist = local.groupby(columns).sum().reset_index(drop=False), tourist.groupby(columns).sum().reset_index(drop=False)

    return local, tourist

In [9]:
def encoding(cate):

    global encoders
    dtypes_cate = cate.dtypes
    encoders = {}
    for column in cate.columns:
        if str(dtypes_cate[column]) == 'object':
            encoder = LabelEncoder()
            encoder.fit(cate[column])
            encoders[column] = encoder

    cate_num = cate.copy()

    for column in encoders.keys():
        encoder = encoders[column]
        cate_num[column] = encoder.transform(cate[column])

    return cate_num

In [10]:
def split(cate_num):
    split_data = []

    X_data, y_data = cate_num.loc[:, cate_num.columns != 'AMT'], cate_num['AMT']
    X_data = X_data.drop(['CSTMR_CNT', 'CNT'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=126, shuffle=True)
    y_train = np.log1p(y_train)
    
    return X_train, X_test, y_train, y_test

In [11]:
# train 데이터 불러오기
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/jeju/201901-202003.csv')

In [12]:
# 시도별 예측 데이터 프레임 생성을 위한 시도 리스트 생성
sido_list = train['CARD_SIDO_NM'].unique().tolist()
sido_list

['강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북']

In [13]:
# sido_list에 저장된 시도들을 이용, for 반복문을 통해 각 시도별 예측 데이터프레임 생성
for sido in sido_list:
  temp = train[train['CARD_SIDO_NM'] == sido].reset_index(drop = True)
  temp = temp.sample(n = 10000).reset_index(drop = True)

  temp = data_pre(temp)
  temp = temp.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

  temp_num = encoding(temp)
  temp_data = split(temp_num)

  temp = Model(temp_data, temp_num)

  temp.get_model()

  temp.final()

  temp_temp = temp.make_temp()

  temp_sub = temp.make_sub(temp_temp)

  globals()['sub_{}'.format(sido)] = temp_sub

[07:24:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:24:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [99]:
# 시도별 2019년 3월 AMT와 2020년 3월 AMT를 비교, 증감율 column을 생성하는 함수

def sido_diff(sido):
  df = train[train['CARD_SIDO_NM'] == sido]
  df_19 = df[df['REG_YYMM'] == 201903]
  df_20 = df[df['REG_YYMM'] == 202003]
  diff = pd.concat([df_19.groupby('STD_CLSS_NM').AMT.sum(), df_20.groupby('STD_CLSS_NM').AMT.sum()], axis=1, keys = ['1903_AMT', '2003_AMT']).reset_index()
  diff = diff.fillna(0)
  diff['diff'] = (diff['1903_AMT']-diff['2003_AMT'])/diff['1903_AMT']
  diff.columns = ['STD_CLSS_NM', '1903_AMT', '2003_AMT', 'diff']
  diff[diff['1903_AMT'] == 0]['diff'] = 0
  diff = diff[['STD_CLSS_NM', 'diff']]
  diff = diff.sort_values(by='diff', ascending=False).reset_index(drop=True)

  return diff

In [100]:
# for문 이용, 각 시도별 증감율 데이터프레임 생성
for sido in sido_list:
  tmp = sido_diff(sido)

  globals()['diff_{}'.format(sido)] = tmp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


생성된 증감율 데이터 프레임을 이용, 기존에 생성해 둔 지역별 예측 값에 증감율을 반영하여 제출하려 했으나, 만든 submission 파일과 정답지의 데이터 형식이 맞지 않아 제출이 되지 않는 현상이 발생.  

확인 결과, 만든 submission의 AMT 컬럼이 float64, 정답지의 AMT 컬럼은 int로, 데이터 타입이 맞지 않음을 확인, int로 변환하려 했으나 두 데이터 프레임에서 문제가 생김.  
  
1. diff_제주 : 2019년 3월 AMT 중 값이 0인 업종이 존재해, 증감율이 -inf가 나오게 됨 → diff 병합 전에 이상치 처리
2. diff_울산 : diff 데이터 프레임 생성 중엔 문제가 없으나, 병합 후 예측치에 존재하는 업종 중 존재하지 않는 업종이 존재, nan값 발생 → 병합 후 결측치 처리


In [108]:
# diff에서 발생하는 이상치(무한대 값) → 결측치로 대체
diff_제주 = diff_제주.replace([np.inf, -np.inf], np.nan)

In [109]:
# 치환한 결측치를 0값으로
diff_제주 = diff_제주.fillna(0)

In [111]:
# 시도별 예측 데이터프레임 하나로 병합
test_subs = [sub_강원, sub_경기, sub_경남, sub_경북, sub_광주, sub_대구, sub_대전, sub_부산, sub_서울, sub_세종, sub_울산, sub_인천, sub_전남, sub_전북, sub_제주, sub_충남, sub_충북]
test_diffs = [diff_강원, diff_경기, diff_경남, diff_경북, diff_광주, diff_대구, diff_대전, diff_부산, diff_서울, diff_세종, diff_울산, diff_인천, diff_전남, diff_전북, diff_제주, diff_충남, diff_충북]

In [112]:
# test_subs와 test_diffs의 카피본 생성
subs = test_subs.copy()
diffs = test_diffs.copy()

In [113]:
# diff값(증감율)을 이용, 예측한 AMT의 값에 (1 - diff)을 곱해서 값을 수정
for i in range(len(sido_list)):
  subs[i] = subs[i].merge(diffs[i][['STD_CLSS_NM','diff']], how='left', on='STD_CLSS_NM')
  subs[i]['AMT'] = subs[i]['AMT']*(1-subs[i]['diff'])
  subs[i] = subs[i][['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]

In [114]:
# 각 시도별로 diff값을 적용시킨 후 하나로 병합 
test = pd.concat(subs)
# 날짜-시도별 순서로 정렬
test = test.sort_values(by = ['REG_YYMM', 'CARD_SIDO_NM']).reset_index(drop = True)

In [115]:
# nan값 확인
test[test['AMT'].isnull()]

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
563,202004,충남,택시 운송업,NaN
1168,202007,충남,택시 운송업,NaN


In [117]:
# inf, -inf값 확인
test[(test['AMT'] == float('inf')) | test['AMT'] == float('-inf')]

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT


In [118]:
# 결측치 → 0으로
test['AMT'] = test['AMT'].fillna(0)

In [119]:
# 결측치 재확인 결과, 존재하지 않음을 확인
test[test['AMT'].isnull()]

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT


In [120]:
# test['AMT']의 데이터 타입을 int로 변환
test['AMT'] = test['AMT'].apply(int)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1210 entries, 0 to 1209
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   REG_YYMM      1210 non-null   int64 
 1   CARD_SIDO_NM  1210 non-null   object
 2   STD_CLSS_NM   1210 non-null   object
 3   AMT           1210 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 37.9+ KB


In [121]:
from google.cloud import bigquery
from tabulate import tabulate
import pandas as pd

project_id = 'jeju-bigquery-282708'
client = bigquery.Client(project = 'jeju-bigquery-282708')

In [123]:
# 제출 파일을 위한 submission 파일 불러오기
submission = client.query('''
  SELECT 
    * 
  FROM `jeju-bigquery-282708.jeju_bigdata.submission` 
  ''').to_dataframe()

In [124]:
# 제출파일 양식에 맞게 가공
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(test, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission['AMT'] = submission['AMT'].fillna(0)
print(submission.head())

   id  REG_YYMM CARD_SIDO_NM           STD_CLSS_NM          AMT
0   0    202004           강원            건강보조식품 소매업  660628559.0
1   1    202004           강원               골프장 운영업  725793968.0
2   2    202004           강원           과실 및 채소 소매업  181397483.0
3   3    202004           강원     관광 민예품 및 선물용품 소매업   31066275.0
4   4    202004           강원  그외 기타 분류안된 오락관련 서비스업          0.0


In [126]:
# submission과 test를 병합 후 float으로 잡힌 AMT 데이터 타입을 int로 변환 후 확인
submission['AMT'] = submission['AMT'].apply(int)
submission.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1394 entries, 0 to 1393
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1394 non-null   int64 
 1   REG_YYMM      1394 non-null   int64 
 2   CARD_SIDO_NM  1394 non-null   object
 3   STD_CLSS_NM   1394 non-null   object
 4   AMT           1394 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 65.3+ KB


In [129]:
# 제출파일 생성
submission.to_csv('/content/drive/My Drive/Colab Notebooks/jeju/submission_region03.csv', encoding='utf-8-sig', index=False)

앞서 제출한 지역별 예측치를 이용하여 얻은 점수(약 4.78)보다 높은 점수(약 3.91)를 얻을 수 있었다.  

적용한 가중변수는 2019년 3월과 2020년 3월의 업종별 AMT의 증감률을 이용한 것으로, 단순 지역별 가중변수와 추가적으로 업종별 가중변수 등을 반영하면 더 좋은 점수를 얻을 수 있을 것이라 생각함.